In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

#models
from RecurrentCNN import R_CNN

#custom Kothadekh
from data_dataloader_v2 import get_loaders
from data_dekhabet import convertTokens2Dekhabet
from Dataset_Control_Toolset import ModelMonitor 
from CreateFilenames import createFileNames

# Initiate Model And Loss

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = R_CNN()
model = model.to(device)

criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True).to(device)


# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0001)


In [ ]:
import torch 

print(torch.__version__)

# Getting The Data

In [ ]:
batch_size= 25
num_epochs = 300

fileNames = createFileNames(3)
rpath = r'data/openSLR/Splitted/Train/'

mm = ModelMonitor(rpath,fileNames,3,0.001,'logs/modelLogs.json')


trainloader, _ = get_loaders(mm.getCurrentDataset(), 
                             split_perc=1,
                             batch_size=batch_size,
                             mode=0, 
                             rootpath=rpath)

curr_Dname =  mm.getCurrentDataset()[0]


# Training

In [ ]:
for epoch in range(num_epochs):
    
    trainiter = iter(trainloader)
    total_step = len(trainloader)
    
    for i in range(total_step-1):
        
        spectros, lbls, lbl_lens = trainiter.next()
        spectros = spectros.to(device)
        
        lbls = lbls.to(device)
        lbl_lens.to(device)
        
        pred = model(spectros)
        preds_size = Variable(torch.LongTensor([pred.size(0)] * batch_size))
        
        #backprop and optimize!
        
        
        optimizer.zero_grad()
        cost = criterion(pred, lbls, preds_size, lbl_lens)/batch_size

        cost.backward()
        optimizer.step()
    
    if (epoch+1) % 100 == 0:
        
        print('Epoch No {}  reached saving model'.format(epoch+1))
        torch.save(model.state_dict(), 'outputModel/KDNet_epoch_rmsProp_{}_test.pkl'.format(epoch+1))
        
        mm.update(cost.item())
        
        if(curr_Dname != mm.getCurrentDataset()[0]):
            
            trainloader, _ = get_loaders(mm.getCurrentDataset(), 
                             split_perc=1,
                             batch_size=batch_size,
                             mode=0, 
                             rootpath=rpath)
            
            curr_Dname = mm.getCurrentDataset()[0]

            
        
        
        

# Running Inference

In [ ]:
inferModel = R_CNN()
inferModel.load_state_dict(torch.load('outputModel/KDNet_epoch_rmsProp_200_noZerolenfixed.pkl'))
inferModel.to(device)

In [ ]:
testIter = iter(validloader)


In [ ]:
spectros, lbls, lbl_lens = testIter.next()

spectros = spectros.to(device)
lbls = lbls.to(device)
lbl_lens.to(device)



preds = inferModel(spectros)
_, preds = preds.max(2)
preds = preds.transpose(1, 0)




print(preds)
print(lbls)

pred_list = preds.transpose(1, 0).contiguous().view(-1)

lbls_list = lbls.contiguous().view(-1)



In [ ]:
print('Our prediction labels: ')

for sent in preds:
    print(convertTokens2Dekhabet(sent))


print('\n\nBelow is the Actual labels: ')
    
    
for sent in lbls:
    print(convertTokens2Dekhabet(sent))

# Validation Inference